In [67]:
import tushare as ts
import pandas as pd
from IPython.display import HTML
stock_selected='600522'

1、线图效果（关系图改造）
======

In [68]:
#历年前十大股东持股情况
#df1为季度统计摘要，data1为前十大持股明细统计
df1, data1 = ts.top10_holders(code=stock_selected, gdtype='0') #gdtype等于1时表示流通股，默认为0 
#df1, data1 = ts.top10_holders(code='002281', year=2015, quarter=1, gdtype='1')

In [69]:
df1 = df1.sort_values('quarter', ascending=True)

In [70]:
df1.tail(10)

,quarter,amount,changed,props
9,2015-11-11,52758.28,19765.96,51.86
8,2015-12-07,51736.39,-1021.89,49.56
7,2015-12-31,44852.98,-6883.41,42.96
6,2016-03-31,44302.75,-550.23,42.43
5,2016-06-30,109362.47,65059.72,41.90
4,2016-09-30,106324.88,-3037.59,40.74
3,2016-12-31,106104.76,-220.12,40.65
2,2017-01-26,105461.76,-643.00,40.40
1,2017-02-08,118482.82,13021.06,38.62
0,2017-03-31,118648.93,166.11,38.67


In [71]:
qts = list(df1['quarter'])
data = list(df1['props'])

In [72]:
lgdstr = """
var axisData = """ + str(qts) + """;
var data = """ + str(data) + """;
var links = data.map(function (item, i) {
    return {
        source: i,
        target: i + 1
    };
});
links.pop();
option = {
    title: {
        text: '【浦发银行】前十大流通股东持股占比'
    },
    tooltip: {
        trigger: 'item'
    },
    xAxis: {
        type : 'category',
        boundaryGap : false,
        data : axisData
    },
    yAxis: {
        type : 'value'
    },
    series: [
        {
            type: 'line',
            layout: 'none',
            coordinateSystem: 'cartesian2d',
            symbolSize: 10,
            label: {
                normal: {
                    show: true
                }
            },
            edgeSymbol: ['circle', 'arrow'],
            edgeSymbolSize: [2, 5],
            data: data,
            links: links,
            lineStyle: {
                normal: {
                    color: '#2f4554'
                }
            }
        }
    ]
};
"""

In [73]:
headstr = """
<div id="showhere" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('showhere'));
"""

tailstr = """
myChart.setOption(option);
    });
</script>
"""

In [74]:
HTML(headstr + lgdstr + tailstr)

2、饼图效果
======

In [75]:
#浦发银行2016三季度前十大流通股东情况
df2, data2 = ts.top10_holders(code=stock_selected, year=2016, quarter=3, gdtype='1')

In [76]:
data2

,quarter,name,hold,h_pro,sharetype,status
20,2016-09-30,中天科技集团有限公司,43308.09,20.08,流通A股,未变
21,2016-09-30,中央汇金资产管理有限责任公司,7247.63,3.36,流通A股,未变
22,2016-09-30,上海浦东发展银行股份有限公司－国泰金龙行业精选证券投资基金,2559.82,1.19,流通A股,新进
23,2016-09-30,银华基金－农业银行－银华中证金融资产管理计划,2162.72,1.00,流通A股,未变
24,2016-09-30,南方基金－农业银行－南方中证金融资产管理计划,2162.72,1.00,流通A股,未变
25,2016-09-30,张家世,1980.13,0.92,流通A股,新进
26,2016-09-30,大成基金－农业银行－大成中证金融资产管理计划,1927.63,0.89,流通A股,未变
27,2016-09-30,中国建设银行股份有限公司－国泰中小盘成长混合型证券投资基金(LOF),1902.41,0.88,流通A股,新进
28,2016-09-30,广发基金－农业银行－广发中证金融资产管理计划,1866.50,0.87,流通A股,未变
29,2016-09-30,工银瑞信基金－农业银行－工银瑞信中证金融资产管理计划,1741.97,0.81,流通A股,新进


In [77]:
#取前十大流通股东名称
top10name = str(list(data2['name']))

In [78]:
valstrs = ''
for idx in data2.index:
    s = '{value: %s, name: \'%s\'}' % (data2.ix[idx]['h_pro'], data2.ix[idx]['name'])
    valstrs += s + ','
valstrs = valstrs[:-1]

In [79]:
datacontent = """
option = {
    tooltip: {
        trigger: 'item',
        formatter: "{a} <br/>{b}: {c} ({d}%)"
    },
    legend: {
        orient: 'vertical',
        x: 'left',
        data: """ + top10name +"""
    },
    series: [
        {
            name:'前十大流通股东：',
            type:'pie',
            radius: ['50%', '70%'],
            avoidLabelOverlap: false,
            label: {
                normal: {
                    show: false,
                    position: 'center'
                },
                emphasis: {
                    show: true,
                    textStyle: {
                        fontSize: '30',
                        fontWeight: 'bold'
                    }
                }
            },
            labelLine: {
                normal: {
                    show: false
                }
            },
            data:[
                """ + valstrs + """
            ]
        }
    ]
};

"""

In [80]:
headstr = """
<div id="mychart" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('mychart'));
"""

tailstr = """
myChart.setOption(option);
    });
</script>
"""

In [81]:
HTML(headstr + datacontent + tailstr)

3、K线效果演示
=========

In [86]:
#中国联通前复权数据
#df = ts.get_k_data(stock_selected, start='2016-01-01', end='2016-12-02')
df = ts.get_k_data(stock_selected, start='2016-01-01')
datastr = ''
for idx in df.index:
    rowstr = '[\'%s\',%s,%s,%s,%s]' % (df.ix[idx]['date'], df.ix[idx]['open'], 
                                           df.ix[idx]['close'], df.ix[idx]['low'], 
                                           df.ix[idx]['high'])
    datastr += rowstr + ','
datastr = datastr[:-1]

In [87]:
#取股票名称
name = ts.get_realtime_quotes(stock_selected)['name'][0]

In [88]:
datahead = """
<div id="chart" style="width:800px; height:600px;"></div> 
<script> 
require.config({ paths:{ echarts: '//cdn.bootcss.com/echarts/3.2.3/echarts.min', } });
require(['echarts'],function(ec){
var myChart = ec.init(document.getElementById('chart'));
"""
datavar = 'var data0 = splitData([%s]);' % datastr
funcstr = """
function splitData(rawData) {
    var categoryData = [];
    var values = []
    for (var i = 0; i < rawData.length; i++) {
        categoryData.push(rawData[i].splice(0, 1)[0]);
        values.push(rawData[i])
    }
    return {
        categoryData: categoryData,
        values: values
    };
}

function calculateMA(dayCount) {
    var result = [];
    for (var i = 0, len = data0.values.length; i < len; i++) {
        if (i < dayCount) {
            result.push('-');
            continue;
        }
        var sum = 0;
        for (var j = 0; j < dayCount; j++) {
            sum += data0.values[i - j][1];
        }
        result.push((sum / dayCount).toFixed(2));
    }
    return result;
}

option = {
    title: {
"""

namestr = 'text: \'%s\',' %name

functail = """
        left: 0
    },
    tooltip: {
        trigger: 'axis',
        axisPointer: {
            type: 'line'
        }
    },
    legend: {
        data: ['日K', 'MA5', 'MA10', 'MA20', 'MA30']
    },
    grid: {
        left: '10%',
        right: '10%',
        bottom: '15%'
    },
    xAxis: {
        type: 'category',
        data: data0.categoryData,
        scale: true,
        boundaryGap : false,
        axisLine: {onZero: false},
        splitLine: {show: false},
        splitNumber: 20,
        min: 'dataMin',
        max: 'dataMax'
    },
    yAxis: {
        scale: true,
        splitArea: {
            show: true
        }
    },
    dataZoom: [
        {
            type: 'inside',
            start: 50,
            end: 100
        },
        {
            show: true,
            type: 'slider',
            y: '90%',
            start: 50,
            end: 100
        }
    ],
    series: [
        {
            name: '日K',
            type: 'candlestick',
            data: data0.values,
            markPoint: {
                label: {
                    normal: {
                        formatter: function (param) {
                            return param != null ? Math.round(param.value) : '';
                        }
                    }
                },
                data: [
                    {
                        name: '标点',
                        coord: ['2013/5/31', 2300],
                        value: 2300,
                        itemStyle: {
                            normal: {color: 'rgb(41,60,85)'}
                        }
                    },
                    {
                        name: 'highest value',
                        type: 'max',
                        valueDim: 'highest'
                    },
                    {
                        name: 'lowest value',
                        type: 'min',
                        valueDim: 'lowest'
                    },
                    {
                        name: 'average value on close',
                        type: 'average',
                        valueDim: 'close'
                    }
                ],
                tooltip: {
                    formatter: function (param) {
                        return param.name + '<br>' + (param.data.coord || '');
                    }
                }
            },
            markLine: {
                symbol: ['none', 'none'],
                data: [
                    [
                        {
                            name: 'from lowest to highest',
                            type: 'min',
                            valueDim: 'lowest',
                            symbol: 'circle',
                            symbolSize: 10,
                            label: {
                                normal: {show: false},
                                emphasis: {show: false}
                            }
                        },
                        {
                            type: 'max',
                            valueDim: 'highest',
                            symbol: 'circle',
                            symbolSize: 10,
                            label: {
                                normal: {show: false},
                                emphasis: {show: false}
                            }
                        }
                    ],
                    {
                        name: 'min line on close',
                        type: 'min',
                        valueDim: 'close'
                    },
                    {
                        name: 'max line on close',
                        type: 'max',
                        valueDim: 'close'
                    }
                ]
            }
        },
        {
            name: 'MA5',
            type: 'line',
            data: calculateMA(5),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA10',
            type: 'line',
            data: calculateMA(10),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA20',
            type: 'line',
            data: calculateMA(20),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },
        {
            name: 'MA30',
            type: 'line',
            data: calculateMA(30),
            smooth: true,
            lineStyle: {
                normal: {opacity: 0.5}
            }
        },

    ]
};
myChart.setOption(option);
    });
</script>
"""

In [89]:
HTML(datahead + datavar + funcstr + namestr + functail)